In [ ]:
import sys
sys.path.append('..')

from src.vrs_utils import vrs_translate
trans = vrs_translate()


from src.hgvs.hgvs_utils import hgvs_translate
trans_hgvs = hgvs_translate()

from src.spdi.spdi_utils import spdi_translate
trans_spdi = spdi_translate()

In [ ]:
vrs = {'_id': 'ga4gh:VA.CxiA_hvYbkD8Vqwjhx5AYuyul4mtlkpD',
 'type': 'Allele',
 'location': {'type': 'SequenceLocation',
  'sequence_id': 'ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl',
  'interval': {'type': 'SequenceInterval',
   'start': {'type': 'Number', 'value': 44908821},
   'end': {'type': 'Number', 'value': 44908822}}},
 'state': {'type': 'LiteralSequenceExpression', 'sequence': 'T'}}

vrs_error = {'_id': 'ga4gh:VA.CxiA_hvYbkD8Vqwjhx5AYuyul4mtlkpD',
 'type': 'ERROR',
 'location': {'type': 'SequenceLocation',
  'sequence_id': 'ga4gh:SQ.IIB53T8CNeJJdUqzn9V_JnRtQadwWCbl',
  'interval': {'type': 'SequenceInterval',
   'start': {'type': 'Number', 'value': 44908821},
   'end': {'type': 'Number', 'value': 44908822}}},
 'state': {'type': 'LiteralSequenceExpression', 'sequence': 'T'}}

In [ ]:
trans.from_vrs_to_spdi(vrs)

In [ ]:
trans.from_vrs_to_normalize_hgvs(vrs)

In [ ]:
trans_hgvs.from_hgvs_to_spdi('NC_000019.10:g.44908822C>T')

In [ ]:
trans_hgvs.from_hgvs_to_vrs('NC_000019.10:g.44908822C>T')

In [ ]:
# Packages imported
import requests
import json

class var_serv_api:

    def __init__(self):
        """ Initialize class with the API URL """
        
        self.base_ncbi_url_api = 'https://api.ncbi.nlm.nih.gov/variation/v0/'

        self.headers = {
            'Content-Type': 'application/json; charset=utf-8'
        }

    def spdi_attribute_concat(self,r):
        """ Extract SPDI attributes, and concatenating the attributes to create a SPDI expressions.

        Args:
            r (request.Response): The response containing SPDI data

        Returns:
            list: A list of SPDI expressions 
        """
        response_json = json.loads(r.text)
        spdi_dict = response_json['data']['spdis'][0] # first spdi expression.
        return ':'.join( (spdi_dict['seq_id'],str(spdi_dict['position']), spdi_dict['deleted_sequence'],spdi_dict['inserted_sequence']) )
    
    def validate_spdi(self,spdi_id):

        endpoint = '/spdi/{}/'.format(spdi_id)
        url = f'{self.base_ncbi_url_api}{endpoint}'
        response = requests.get(url,headers=self.headers)

        if response.status_code == 200:

            response_json = json.loads(response.text)
            spdi_dict = response_json['data']
            return ':'.join( (spdi_dict['seq_id'],str(spdi_dict['position']), spdi_dict['deleted_sequence'],spdi_dict['inserted_sequence']) )
        else:
            try:
                error_json = response.json()
                error_message = error_json['error']['message']
                raise requests.HTTPError(f'Failed to validate SPDI expression: {spdi_id}. Error Message: {error_message}')
            except json.JSONDecodeError:
                raise requests.HTTPError(f'Failed to parse error response as JSON: {response.text}')
            # try:
            #     error_json = response.json()
            #     error_message = error_json['error']['message']
            # except (json.JSONDecodeError, KeyError):
            #     error_message = response.text
            # raise requests.HTTPError(f'Failed to validate SPDI expression: {spdi_id}. Error Message: {error_message}')
    #NOTE:OLD -- compare these two later.
    # def validate_spdi(self,spdi_id):

    #     endpoint = '/spdi/{}/'.format(spdi_id)
    #     url = f'{self.base_ncbi_url_api}{endpoint}'
    #     response = requests.get(url,headers=self.headers)

    #     if response.status_code == 200:

    #         response_json = json.loads(response.text)
    #         spdi_dict = response_json['data']
    #         return ':'.join( (spdi_dict['seq_id'],str(spdi_dict['position']), spdi_dict['deleted_sequence'],spdi_dict['inserted_sequence']) )
    #     else:
    #         raise requests.HTTPError(f'Request failed with status code: {response.status_code}')
    
    def spdi_to_hgvs(self,spdi_id):
        """ Translate SPDI expression to right-shift hgvs notation. 
        End point used from variation service api: /spdi/{spdi}/hgvs

        Args:
            spdi_id (str): SPDI expression 

        Raises:
            requests.HTTPError: If the request fails with a non-200 status code.

        Returns:
            str: Right shift normalized HGVS representation of the SPDI expression.
        """
        endpoint = '/spdi/{}/hgvs'.format(spdi_id)
        
        url = f'{self.base_ncbi_url_api}{endpoint}'

        response = requests.get(url,headers=self.headers)

        if response.status_code == 200:
            return json.loads(response.text)['data']['hgvs']
        else:
            raise requests.HTTPError(f'Request failed with status code: {response.status_code}')

    def hgvs_to_spdi(self,hgvs_id, assembly ='GCF_000001405.38'):
        """Translate HGVS expression to SPDI expression.
        End point used from variation service api: /hgvs/{hgvs}/contextuals

        Args:
            hgvs_id (str): HGVS expression
            assembly (str, optional): Assembly version. Defaults assemlby version to 'GCF_000001405.38'.

        Raises:
            requests.HTTPError: If the request fails with a non-200 status code.

        Returns:
            str: The SPDI representation of the HGVS expression. 
        """

        endpoint = '/hgvs/{}/contextuals{}'.format(hgvs_id,assembly)
        
        url = f'{self.base_ncbi_url_api}{endpoint}' 

        response = requests.get(url, headers=self.headers)

        if response.status_code == 200:
            return self.spdi_attribute_concat(response)
        else:
            raise requests.HTTPError(f'Request failed with status code: {response.status_code}')





In [ ]:
test = var_serv_api()
test.validate_spdi('NC_000001.1012345:0:C')


In [ ]:
test.hgvs_to_spdi('NC_000001.10:g.12345T>A')

In [ ]:
trans_spdi.from_spdi_to_rightshift_hgvs('NC_000001.11:935845:GCCGC:GCCGCCGC')

In [ ]:
trans_spdi.from_spdi_to_vrs('NC_000001.11:935845:GCCGC:GCCGCCGC')

In [ ]:
# from src.ncbi_variation_services_api import var_serv_api
# from ga4gh.vrs.extras.translator import Translator
# from ga4gh.vrs.dataproxy import SeqRepoRESTDataProxy


class SPDI:
    def __init__(self,sequenceId,position,deletedSequence,insertedSequence):
        self.sequenceId = sequenceId
        self.position = position
        self.deletedSequence = deletedSequence
        self.insertedSequence = insertedSequence

    def __str__(self):
        return f"{self.sequenceId}:{self.position}:{self.deletedSequence}:{self.insertedSequence}"
    
    # def to_string(self):
    #     return f"{self.sequenceId}:{self.position}:{self.deletedSequence}:{self.insertedSequence}"
    
    def to_dict(self):
        return {
            'sequenceID': self.sequenceId,
            'position': self.position,
            'deletedSequence': self.deletedSequence,
            'insertedSequence': self.insertedSequence
        }

class spdi_translate:

    def __init__(self):

        self.seqrepo_rest_service_url = "https://services.genomicmedlab.org/seqrepo"
        self.dp = SeqRepoRESTDataProxy(base_url=self.seqrepo_rest_service_url)
        self.tlr = Translator(data_proxy=self.dp,
                 translate_sequence_identifiers=True,  # default
                 normalize=True,                       # default
                 identify=True)                        # default
        self.var_serv_api = var_serv_api()

    def from_spdi_to_rightshift_hgvs(self,expression):
        """ Translate SPDI expression to right-shift HGVS expression. (Using NCBI API)

        Args:
            expression (string): SPDI expression 

        Returns:
            str: Right shift normalized HGVS expressions
        """

        spdi_expression = self.var_serv_api.validate_spdi(expression)

        try: 
            return self.var_serv_api.spdi_to_hgvs(spdi_expression)
        except Exception as e: 
            return '{}. Expression Error: {}'.format(e,spdi_expression)

#TODO: Write documentation
    def from_spdi_to_vrs(self,expression):
        """SPDI --> VRS

        Args:
            expression (_type_): _description_

        Returns:
            _type_: _description_
        """
        
        spdi_expression = self.var_serv_api.validate_spdi(expression)

        try:
            vrs_expression = self.tlr.translate_from(spdi_expression,"spdi")
            return vrs_expression.as_dict()
        except Exception as e:
            return '{}. Expression Error: {}'.format(e,expression) 


spdi_expression = SPDI(sequenceId="NC_000001.11",position = 935845,deletedSequence="GCCGC",insertedSequence="GCCGCCGC")


type(spdi_expression)
print(spdi_expression)
# print(spdi_expression)
# print(spdi_expression.to_string())
# print(spdi_expression.to_dict())

In [ ]:
spdi_instance = SPDI('refseq:NC_000001.11',100,'A','T')
spdi_instance
# Then, create an instance of the spdi_translate class
translator = spdi_translate()

# Now, use the from_spdi_to_vrs method to translate the SPDI expression to VRS format
vrs_result = translator.from_spdi_to_vrs(spdi_instance.to_dict())

# The vrs_result will contain the VRS expression in dictionary format
print(vrs_result)

In [ ]:
spdi_instance

In [ ]:
from src.spdi.spdi_class import SPDI

spdi = SPDI(sequence="NC_000001.11", position=930328, deletion="AA", insertion="")
print(spdi)
print(type(spdi.to_string()))
print(spdi.to_dict())

In [ ]:
from src.api.ncbi_variation_services_api import var_serv_api
from ga4gh.vrs.extras.translator import Translator
from ga4gh.vrs.dataproxy import SeqRepoRESTDataProxy

import hgvs.parser

#TODO: Class names should follow a camelCase 
class spdi_translate:

    def __init__(self):

        self.seqrepo_rest_service_url = "https://services.genomicmedlab.org/seqrepo"
        self.dp = SeqRepoRESTDataProxy(base_url=self.seqrepo_rest_service_url)
        self.tlr = Translator(data_proxy=self.dp,
                 translate_sequence_identifiers=True,  # default
                 normalize=True,                       # default
                 identify=True)                        # default
        self.var_serv_api = var_serv_api()
        self.hp = hgvs.parser.Parser()

#TODO: reWrite documentation 
    def from_spdi_to_rightshift_hgvs(self,expression,validate= True,format_output= True):
        """ Translate SPDI expression to right-shift HGVS expression. (Using NCBI API)

        Args:
            expression (string): SPDI expression 

        Returns:
            str: Right shift normalized HGVS expressions
        """
        if validate:
            spdi_expression = self.var_serv_api.validate_spdi(expression)
        else:
            spdi_expression = expression

        try: 
            if format_output:
                return self.var_serv_api.spdi_to_hgvs(spdi_expression)
            else:
                hgvs_expression = self.var_serv_api.spdi_to_hgvs(spdi_expression)
                return self.hp.parse_hgvs_variant(hgvs_expression)
        except Exception as e: 
            return '{}. Expression Error: {}'.format(e,spdi_expression)

#TODO: reWrite documentation
    def from_spdi_to_vrs(self,expression,validate= True):
        """SPDI --> VRS

        Args:
            expression (_type_): _description_

        Returns:
            _type_: _description_
        """
        
        if validate:
            spdi_expression = self.var_serv_api.validate_spdi(expression)
        else:
            spdi_expression = expression

        try:
            vrs_expression = self.tlr.translate_from(spdi_expression,"spdi")
            return vrs_expression.as_dict()
        except Exception as e:
            return '{}. Expression Error: {}'.format(e,expression) 


In [ ]:
test = spdi_translate()

In [ ]:
test.from_spdi_to_rightshift_hgvs('NC_000001.10:150550915:GACAA:CAATACC',format_output = False)

In [ ]:
import hgvs.parser
par = hgvs.parser.Parser()

In [ ]:
par.parse_hgvs_variant('NC_000001.10:g.150550916_150550920delinsCAATACC')

In [3]:
import re 

class CoreVariantClass:
    
    """ The CoreVariantClass is draft schema."""

    def __init__(self,origCoordSystem,seqType):
        self._original_attrs = {'cord':origCoordSystem,'seqType':seqType}
        self.origCoordSystem = origCoordSystem
        self.seqType = seqType


        self._validate_origCoordSystem()
        self._validate_seqType()


        
        self.initParamValues ={
                'origCoordSystem': self.origCoordSystem,
                'seqType':  self.seqType}
    def initParams(self):
        """ A dictionary of the initial parameters. 

        Returns:
            dict: A dictionary (Key: Initial parameters, Values: parameters values). 
        """
        return self._original_attrs


    def _validate_origCoordSystem(self):
        """ Validate origCoordSystem input. Method checks if the provided origCoordSystem
        is one of the allowed types: ('0-based interbase','0-based counting','1-based counting').

        Args:
            coordSystem (str): The origCoordSystem to be validated.

        Raises:
            ValueError: If the provided input is not one of the allowed types.

        Returns:
            str: One of the allowed types. 
        """
        value = self.origCoordSystem.strip()
        allowedOrigCoordSystem = ('0-based interbase','0-based counting','1-based counting')
        if value not in allowedOrigCoordSystem:
            raise ValueError(f'Invalid origCoordSystem input: "{self.origCoordSystem}". Allowed types: {allowedOrigCoordSystem}.')
        self.origCoordSystem = value
    
    def _validate_seqType(self):
        """ Validate seqType input. Method checks if the provided seqType is one of the allowed types: ('DNA','RNA','PROTEIN'). 

        Args:
            seqType (str): The seqType to be validated. 

        Raises:
            ValueError: If the provided input is not one of the allowed types. 

        Returns:
            str: One of the allowed types in uppercase. 
        """
        value = self.seqType.upper().strip()
        allowedSeqType = ('DNA','RNA','PROTEIN')
        if value not in allowedSeqType:
            raise ValueError(f'Invalid seqType input: "{self.seqType}". Allowed types: {allowedSeqType} (Case Insensitive).') 
        self.seqType = value

In [4]:
# Example usage
orig_coord_system = '0-based interbase'
seq_type = 'dna'
obj = CoreVariantClass(orig_coord_system, seq_type)

# Accessing original and validated values
print("Original attributes:", obj.initParams())
print("Validated attributes:", obj.origCoordSystem, obj.seqType)


Original attributes: {'cord': '0-based interbase', 'seqType': 'dna'}
Validated attributes: 0-based interbase DNA
